In [3]:
import os
import sys
import tqdm
import random
import numpy as np
import scipy.stats
from statsmodels.distributions.empirical_distribution import ECDF

TEST_FRACTION = 0.2

def js_distance(P, Q):
    overall_min = min(P.min(), Q.min())
    overall_max = max(P.max(), Q.max())
    kP = scipy.stats.gaussian_kde(P)
    kQ = scipy.stats.gaussian_kde(Q)
    xs = np.linspace(overall_min, overall_max, 100)
    Psmooth = kP(xs)
    Qsmooth = kQ(xs)
    Msmooth = (Psmooth + Qsmooth) / 2
    KL_PM = scipy.stats.entropy(Psmooth, Msmooth)
    KL_QM = scipy.stats.entropy(Qsmooth, Msmooth)
    return 0.5 * (KL_PM + KL_QM)

data_dir = os.path.normpath(os.path.join(sys.path[0], '../data/'))
basenames = os.listdir(os.path.join(data_dir, 'interim', 'X'))
num_test = int(len(basenames) * TEST_FRACTION)
num_train = len(basenames) - num_test
labels = ['test'] * num_test + ['train'] * num_train

# Pick a train-test split such that the training and testing distributions look
# as similar as possible. With a much larger amount of data, this would be
# unnecessary; however, there are only a fairly small number of seizures, and so
# we can't necessarily rely on random sampling alone.

distances = []
for i in tqdm.trange(1):
    random.shuffle(basenames)
    Ys_clustered = {'train': [], 'test': []}
    for (kind, file) in zip(labels, basenames):
        Ys_original = np.load(os.path.join(data_dir, 'interim', 'Y', file))
        Ys_clustered[kind] += list(Ys_original)
    train_dist = np.array(Ys_clustered['train'])
    test_dist = np.array(Ys_clustered['test'])
    distances.append((js_distance(train_dist, test_dist), basenames))

basenames = min(distances)[1]

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


In [8]:
from IPython.core.debugger import set_trace

In [4]:
all_features = np.concatenate([
    np.load(os.path.join(data_dir, 'interim', 'X', basename))
    for basename in basenames
])

In [5]:
cdfs = [ECDF(values) for values in all_features.T]

In [13]:
def normalize_feature_vectors(Xs_original):
    Xs_transformed = np.full_like(Xs_original, np.nan)
    for (i, (x, ecdf)) in enumerate(zip(Xs_original.T, cdfs)):
        # The nudging is to prevent the highest/lowest value from getting a
        # quantile of exactly +/- 1, which would have a z-score of +/- infinity
        x_nudged = x + 0.000000001 * np.ptp(x) * np.sign(np.median(x) - x)
        x_transformed = scipy.stats.norm.ppf(ecdf(x_nudged))
        if not np.isfinite(x_transformed).all():
            set_trace()
        Xs_transformed[:, i] = x_transformed
    return Xs_transformed

In [14]:
for (kind, file) in zip(labels, basenames):
    Xs_original = np.load(os.path.join(data_dir, 'interim', 'X', file))
    Ys_original = np.load(os.path.join(data_dir, 'interim', 'Y', file))
    Xs_normalized = normalize_feature_vectors(Xs_original)
    Xs_normalized = np.concatenate([
        Xs_normalized[:, :585],
        Xs_normalized[:, 603:639],
        Xs_normalized[:, 640:]
    ], axis=1)
    print(file)

chb12_42_0.npy
chb18_31_0.npy
chb14_18_0.npy
chb13_19_0.npy
chb18_30_0.npy
chb23_08_1.npy
chb19_28_0.npy
chb12_42_3.npy
chb13_40_0.npy
chb12_42_4.npy
chb06_18_0.npy
chb06_04_0.npy
chb18_32_0.npy
chb04_28_0.npy
chb03_01_0.npy
chb15_40_0.npy
chb04_08_0.npy
chb05_22_0.npy
chb02_19_0.npy
chb10_89_0.npy
chb10_12_0.npy
chb12_38_2.npy
chb06_01_0.npy
chb20_13_1.npy
chb23_09_1.npy
chb08_13_0.npy
chb13_40_1.npy
chb17a_04_0.npy
chb15_54_4.npy
chb22_38_0.npy
chb15_31_0.npy
chb09_06_0.npy
chb15_49_0.npy
chb12_08_2.npy
chb10_27_0.npy
chb04_05_0.npy
chb13_55_0.npy
chb09_19_0.npy
chb12_38_3.npy
chb21_19_0.npy
chb15_28_0.npy
chb08_05_0.npy
chb12_38_1.npy
chb12_08_3.npy
chb12_09_1.npy
chb22_25_0.npy
chb01_04_0.npy
chb23_08_0.npy
chb08_02_0.npy
chb16_10_0.npy
chb11_99_0.npy
chb13_60_0.npy
chb19_29_0.npy
chb09_08_1.npy
chb21_21_0.npy
chb17a_03_0.npy
chb13_62_0.npy
chb20_16_0.npy
chb13_59_0.npy
chb02_16+_0.npy
chb06_13_0.npy
chb06_09_0.npy
chb19_30_0.npy
chb12_06_1.npy
chb03_04_0.npy
chb15_40_2.npy
chb05_1